# Introduction
This tutorial covers how to get started with your own Julia package. It assumes some familiarity with the language and that you have a working version of Julia installed on your computer. We also highly recommend installing the [Juno IDE](http://junolab.org/).

This tutorial is built for Julia v1.0.0+.

# Getting Started

## Getting Packages
Julia v1.0 has a brand-new package manager with a lot of 
functionality. We'll cover some of the basics here.

### Using the Package Manager
To use the package manager from the REPL (recommended) press the `]` key.

### File Locations
If you installed Julia with default settings, it should place all files beneath your home directory:

`/home/username/.julia`

A couple of important sub-directories:
* `dev` - contains git repositories of the packages being developed. It is okay to edit files in this directory
* `packages` - contains git repositories of the packages downloaded via the package manager. You should not edit these files.
* `compiled` - contains compiled files (`*.ji`) for pre-compiled packages.


### Adding a registered package
To add a registered package use the `add` command within the Package Manager. This will install the package on your computer and add it to your active project. For example, to add the ForwardDiff package simple enter

`(v1.0) pkg> add ForwardDiff`


### Using a Package
Once installed, a package is brought into the the workspace using the `using` command:


In [10]:
using ForwardDiff

## Creating your Own Package
1. Go to GitHub.com and create a new repository. By convention it should end with `*.jl` (i.e. `NewPackage.jl`). Make sure you create a blank repo (no README or .gitignore or license)
![Add_GitHub_Repo](../img/AddRepo.png)

2. Generate the package locally by opening the Julia REPL and entering
```juila
(v1.3) pkg> generate NewPackage
```

3. Link your files to your GitHub repo (these instructions are also listed on the GitHub page of your empty repo):
```shell
git init
git add -A
git commit -m "initial commit"
git remote add origin https://github.com/username/NewPackage.jl.git
git push -u origin master
```

2. Add new repository to developed Julia packages using the Package Manager
```julia
(v1.0) pkg> dev https://github.com/username/NewPackage.jl.git
```

3. (optional) Delete out the temporary repo you created in step 2

In [7]:
# <src/NewPackage.jl 
module NewPackage
    greet() = "Hi there, welcome to NewPackage!"
end

Main.NewPackage

## Using your Package
### REPL
To use your new package from the REPL simply enter 
```julia
using NewPackage
NewPackage.greet()
```

### Juno
The process is identical, simply enter the previous commands into the Julia Console. 
To open the console go to Julia|Open Console, enter (CTRL+J,CTRL+O), or `Julia Client: Open Console` from the Command Pallette (CTRL+SHIFT+P)

# Using Different Files
Typically the main project file does not contain source code; instead, it collects code from other files and takes care of exporting critical functions. We'll cover all of that here. First, we will cover the `include` function.

## `Include` function
The `include` function essentially copies the code from a source file into another. Let's say I have a file with the following contents:

```julia
# <test_include.jl>
myfun(x) = x + 2 + e^x
a = 4
b = c + 2;
```

If we now use `include("test_include.jl")` it will run the contents of the file:

In [1]:
include("test_include.jl")

LoadError: UndefVarError: c not defined

Notice we get an error because the variable `c` is not defined. If we define `c` and call include again we get the expected behavior

In [10]:
c = 5
include("test_include.jl")
@show a
@show b
@show myfun(2);

a = 4
b = 7
myfun(2) = 8


In practice, we use this to "include" files into the main project file. Typically, these file do not contain anything except functions (and possibly sub-modules). For example, let's say we want to add a new function to our project, located in `project_funtions.jl`

In [13]:
# <src/project_functions.jl>
newfun(x::Int) = x + 3
newfun(x::Float64) = x + π
newfun(x::String) = x * "π";

We now include this in our main project file

In [1]:
# <src/NewProject.jl>
module NewProject
    include("project_functions.jl")
    greet() = "Hi there, welcome to NewPackage!"
end

Main.NewProject

These functions are now part of our package. Test them out by opening up a **new** REPL (or restarting it inside Juno) and enter 
```julia
using NewPackage
```

In [3]:
# Note the use of multiple dispatch!
@show NewProject.newfun(1)
@show NewProject.newfun(1.0)
@show NewProject.newfun("1");

NewProject.newfun(1) = 4
NewProject.newfun(1.0) = 4.141592653589793
NewProject.newfun("1") = "1π"


## Exporting Functions
Note that we have to, rather inconveniently, prepend "MyPackage" to each of our function calls. To avoid this, we can export them. Exporting a function will allow the function to be used by itself in whatever scope into which it is imported. Note that we cannot replicate this same behavior here in a Jupyter notebook since we aren't actually manipulating packages here, simply creating modules.

In [ ]:
# <src/NewProject.jl>
module NewProject
    export
        newfun,
        greet
    
    include("project_functions.jl")
    greet() = "Hi there, welcome to NewPackage!"
end

## Importing Functions
The `import` function has two primary uses:
1. Importing a function into the local scope to avoid prepending the package name every time you want to use it
2. Adding additional methods to a function

### Adding Function methods
In Julia terminology, a function is the name of the function, and method is an instantiation of a function for a particular set of types. The `+` function, for examples, a just a single function, but has many, many methods. You can enumerate them with the `methods` function:

In [11]:
methods(+)

# 168 methods for generic function "+":
[1] +(x::Bool, z::Complex{Bool}) in Base at complex.jl:282
[2] +(x::Bool, y::Bool) in Base at bool.jl:96
[3] +(x::Bool) in Base at bool.jl:93
[4] +(x::Bool, y::T) where T<:AbstractFloat in Base at bool.jl:104
[5] +(x::Bool, z::Complex) in Base at complex.jl:289
[6] +(a::Float16, b::Float16) in Base at float.jl:398
[7] +(x::Float32, y::Float32) in Base at float.jl:400
[8] +(x::Float64, y::Float64) in Base at float.jl:401
[9] +(z::Complex{Bool}, x::Bool) in Base at complex.jl:283
[10] +(z::Complex{Bool}, x::Real) in Base at complex.jl:297
[11] +(::Missing, ::Missing) in Base at missing.jl:114
[12] +(::Missing) in Base at missing.jl:100
[13] +(::Missing, ::Number) in Base at missing.jl:115
[14] +(level::Base.CoreLogging.LogLevel, inc::Integer) in Base.CoreLogging at logging.jl:106
[15] +(c::BigInt, x::BigFloat) in Base.MPFR at mpfr.jl:413
[16] +(a::BigInt, b::BigInt, c::BigInt, d::BigInt, e::BigInt) in Base.GMP at gmp.jl:503
[17] +(a::BigInt, b::BigInt, c::BigInt, d::BigInt) in Base.GMP at gmp.jl:502
[18] +(a::BigInt, b::BigInt, c::BigInt) in Base.GMP at gmp.jl:501
[19] +(x::BigInt, y::BigInt) in Base.GMP at gmp.jl:472
[20] +(x::BigInt, c::Union{UInt16, UInt32, UInt64, UInt8}) in Base.GMP at gmp.jl:509
[21] +(x::BigInt, c::Union{Int16, Int32, Int64, Int8}) in Base.GMP at gmp.jl:515
[22] +(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat, e::BigFloat) in Base.MPFR at mpfr.jl:563
[23] +(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat) in Base.MPFR at mpfr.jl:556
[24] +(a::BigFloat, b::BigFloat, c::BigFloat) in Base.MPFR at mpfr.jl:550
[25] +(x::BigFloat, c::BigInt) in Base.MPFR at mpfr.jl:409
[26] +(x::BigFloat, y::BigFloat) in Base.MPFR at mpfr.jl:378
[27] +(x::BigFloat, c::Union{UInt16, UInt32, UInt64, UInt8}) in Base.MPFR at mpfr.jl:385
[28] +(x::BigFloat, c::Union{Int16, Int32, Int64, Int8}) in Base.MPFR at mpfr.jl:393
[29] +(x::BigFloat, c::Union{Float16, Float32, Float64}) in Base.MPFR at mpfr.jl:401
[30] +(x::Dates.CompoundPeriod, y::Dates.CompoundPeriod) in Dates at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.3/Dates/src/periods.jl:343
[31] +(x::Dates.CompoundPeriod, y::Dates.Period) in Dates at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.3/Dates/src/periods.jl:341
[32] +(x::Dates.CompoundPeriod, y::Dates.TimeType) in Dates at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.3/Dates/src/periods.jl:371
[33] +(x::Dates.Date, y::Dates.Day) in Dates at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.3/Dates/src/arithmetic.jl:74
[34] +(x::Dates.Date, y::Dates.Week) in Dates at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.3/Dates/src/arithmetic.jl:72
[35] +(dt::Dates.Date, z::Dates.Month) in Dates at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.3/Dates/src/arithmetic.jl:55
[36] +(dt::Dates.Date, y::Dates.Year) in Dates at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.3/Dates/src/arithmetic.jl:28
[37] +(dt::Dates.Date, t::Dates.Time) in Dates at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.3/Dates/src/arithmetic.jl:19
[38] +(t::Dates.Time, dt::Dates.Date) in Dates at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.3/Dates/src/arithmetic.jl:20
[39] +(x::Dates.Time, y::Dates.TimePeriod) in Dates at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.3/Dates/src/arithmetic.jl:78
[40] +(dt::Dates.DateTime, z::Dates.Month) in Dates at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.3/Dates/src/arithmetic.jl:48
[41] +(dt::Dates.DateTime, y::Dates.Year) in Dates at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.3/Dates/src/arithmetic.jl:24
[42] +(x::Dates.DateTime, y::Dates.Period) in Dates at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.3/Dates/src/arithmetic.jl:76
[43] +(B::BitArray{2}, J::LinearAlgebra.UniformScaling)

Say you have a new type for which you want to define addition, we can do this by either:
a) importing the function, or 
b) Specifying the parent module
For example:

In [10]:
struct MyType
    a::Float64
    b::Float64
end
Base.:+(x::MyType, y::MyType) = MyType(x.a + y.a, x.b + y.b)

# or
import Base: +
+(x::MyType, c::Real) = MyType(x.a + c, x.b)

+ (generic function with 168 methods)

# Workflow

You may have already noticed that we requested you to reboot the REPL after each time you changed your package. This is necessary due to the way Julia handles packages within the global scope of the REPL. There are two ways of getting around this development headache:

1. Revise.jl
2. Juno IDE

### Revise.jl
Revise.jl is made specifically to get around this issue of having the reload the workspace when changes are made to a package/module. Basically you run it prior to loading in any other modules and it takes of most of the headache. 

### Juno IDE (recommended)
Juno has some clever ways to get around this issue, and has some other nice features that make your life as a developer easier. Here we will go over basic workflow to minimize how many times you need to restart the REPL.

1. Create a "working" script that acts basically like your REPL. You easily turn these scripts into test scripts later on.

2. Change the "working module" of the script by clicking "Main" in the status bar and enter the name of your package, or use the shortcut (CTRL+J CTRL+M). The status bar should change to display the name of you module once you do this. Note that this is file-dependent. You should notice that Juno does this automatically for files that are included in your main file. You can even do this for the REPL to access the variables in the workspace of your module.
![status bar](../img/Juno_status.png)
![change_module](../img/ChangeModule.png)

3. With the active module set to your package, any variables, functions, types, etc. that you create in your scipt will be created in the workspace of the your package, and has direct access to all variables and functions of your package (i.e. you don't need to use NewPackage.newfun, even if you haven't exported it). 

4. If you add or change a function in one of the files included in your project, you can reload it by hitting CTRL-RETURN (moves to next statement or function) or SHIFT-RETURN (doesn't move cursor). You can now re-run that function in your "working" script and will see the change.

5. DISCLAIMER: This will save you a lot of time, but it often a good idea to restart your REPL if you begin to observe odd behavior. It's not uncommon to have a variable, function, or method that you don't realize is there that affects behavior. When in doubt: restart the REPL. NOTE: You will ALWAYS needs to restart the REPL after making changes to custom types. 

### Example
Let's do this for our `newfun` function.
* Restart the REPL and enter `using NewPackage`. Open a new file (CTRL-N) named `newfun_test.jl`:
```julia
# <src/newfun_test.jl
newfun("1")
```

* Change the active workspace of `newfun_test.jl` to `NewPackage` using CTRL-J CTRL-M.

* With your cursor on the line testing `newfun`, hit CTRL-RETURN and you'll see the result pop up beside it:
![newfun_test](../img/newfun_test.png)

* Now open `src/project_functions.jl` in Juno and change the string operation to include a space. Afterwards hit CTRL-RETURN to update the function. You should the name of the function pop up beside it.
![newfun_change](../img/newfun_change.png)

* Return to `newfun_test.jl` and rerun the test using CTRL-RETURN. You should see the change reflected, without having to restart the REPL.
![newfun_updated](../img/newfun_updated.png)

## Jupyter Notebooks
Some people like working in Jupyter Notebooks and then migrate code into Juno. We highly discourage this for several reasons:
* With code in a Jupyter Notebook you have to restart the kernel every time you make changes to a package you're importing. Restarting Jupyter kernels take FOREVER!
* Notebooks don't accurately replicate the environment of working on a package
* Given the advantages of Juno IDE, it doesn't make sense to develop in an environment other than the one you'll ultimately be using
* That being said, Jupyter notebooks are great for prototyping ideas in an isolated environment from the rest of your code, and for tutorials (like this one), and for showcasing your project after your codebase has stabilized (i.e. demo notebooks).

# Testing

You should include unit tests as you develop your package. Include all tests in `test/runtests.jl`.
See [documentation](https://docs.julialang.org/en/v1/stdlib/Test/index.html) for details on unit testing. We give a brief example below. 


### Running Tests
You can run all tests for your package from the Package Manager:
```
(v1.0) pkg> test NewPackage
```
Or by calling the function directly from the package:
```julia
include("test/runtests.jl")
```
Or by opening the file in Juno and hitting CTRL-SHIFT-RETURN

You should always run these tests in the Main workspace, to replicate the actual testing environment. However, for debugging it is often useful to have the test suites in separate files that are included in `test/runtests.jl` via `include`. You change the workspace of these files to debug and modify your code while you work on it. 

### Travis CI
[Travis CI](https://travis-ci.org/) is a tool for automatically testing your package. A badge indicating the status of your build is usually displayed in your README.md on GitHub, and lends credibility to your project. Setting up a project with Travis CI is straight-forward, got to https://travis-ci.org/ and sign up with your GitHub account. Simply follow the prompts to set up Travis CI with your repository. You'll then need to create a `.travis.yml` file in your root directory. Here is an example that also sends results to some code coverage utilities:
```yml
## Documentation: http://docs.travis-ci.com/user/languages/julia/
language: julia
os:
  - linux
  - osx
julia:
  - 1.3
  # - nightly

codecov: true

```

Once you push the `.travis.yml` file to GitHub, it should automatically trigger a build on Travis, if you have it set up on your user or organization account (it's automatically activated for all repos in the lab organization).

### Code Coverage
You can also set up code coverage reports with https://codecov.io/. Go to https://codecov.io/gh/RoboticExplorationLab/NewPackage.jl (or whatever your repo name is) and copy the token that appears on the screen into `.codecov.yml` like this:
```yml
comment: false
codecov:
  token: aaaaaaaa-aa11-1a11-aa11-11a11a111111 
```

# Performance Tips
See the [Julia documentation](https://docs.julialang.org/en/v1/manual/performance-tips/index.html) for a complete collection of performance tips. Here we summarize some of the tips we found most useful and ones we use frequently during development

1. Use @btime from BenchmarkTools.jl. This macro is used the same as @time but runs many iterations and give a MUCH more reliable and consistent measure of time. Be advised that it consequently takes more time to run. We often find it very helpful to have a REPL, Jupyter notebook, or separate file open for just making timing comparisons. What you may think is the fastest way to do something may not actually be the case. The Julia comipler is a black box of magic that sometimes does suprising things (especially when you're new to it).

2. Using the Juno IDE, import the Juno repository with `using Juno` which gives you access to `@profiler`. Use this macro before a line of code you want to profile for performance. Juno will subsequently open an interactive plot of the profile results (picture from https://docs.junolab.org/latest/man/juno_frontend.html). If you get an error saying "No profile data" your function execution time is too short to profile.
![profiler from Juno website](../img/profiler.png)

3. Type stability can have major effects on performance. Use @code_warntype to evaluate code stability. 